In [1]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np

#import action chains
from selenium.webdriver.common.action_chains import ActionChains

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

%load_ext autoreload
%load_ext line_profiler
%autoreload 2


import airbnb_fetcher as airbnb

import os

# Average prices

In [ ]:
df_dates = pd.read_excel('Data/TaylorSwiftDates.xlsx')

options = webdriver.ChromeOptions()
service = Service(ChromeDriverManager().install())

df_new = pd.DataFrame(columns=['Cities', 'Day_pre', 'Day_after', 'house_link', 'price'])
def get_row_prices(x):
    global df_new

    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://www.airbnb.dk/?_set_bev_on_new_domain=1662555873_ZGI4NGUwYWE2Zjkw")
    driver.maximize_window()

    print(f"Getting prices for {x['Cities']} on {x['Day_pre'].date()} to {x['Day_after'].date()}")

    time_start = time.time()

    date_pre = str(x['Day_pre']).split(' ')[0]
    date_after = str(x['Day_after']).split(' ')[0]

    price = airbnb.get_city_prices_avg(driver, x['Cities'], date_pre,	date_after) 

    df = pd.DataFrame(data = {'Cities': [x['Cities']], 'Day_pre': [date_pre], 'Day_after': [date_after], 'price': [price]})
    
    df_new = pd.concat([df_new, df])

df_dates.apply(get_row_prices, axis=1)

#df_dates['avg_price'] = df_dates.apply(get_row_prices, axis=1)
for i in list(df_dates.index):
    get_row_prices.iloc[i,:]

df_new.to_csv('Data/airbnb_prices.csv')

# Threaded: prices and features per stay

In [ ]:
df_dates = pd.read_excel('TaylorSwiftCross.xlsx')
i_df = list(df_dates.index)

options = webdriver.ChromeOptions()
service = Service(ChromeDriverManager().install())

def get_row_prices(x, A):
    driver = webdriver.Chrome(service=service, options=options)
    #driver = webdriver.Chrome()
    driver.get("https://www.dr.dk/")
    driver.maximize_window()

    # delete cookies
    driver.delete_all_cookies()
    time.sleep(2)

    #driver.maximize_window()

    try:
        print(f"Getting prices for {x['Cities']} on {x['Day_pre'].date()} to {x['Day_after'].date()}")

        time_start = time.time()

        date_pre = str(x['Day_pre']).split(' ')[0]
        date_after = str(x['Day_after']).split(' ')[0]

        _links, _info, location, rating, price, _features  = airbnb.get_houses_and_infoThreaded(driver, x['Cities'], date_pre,	date_after) 


        df_here = pd.DataFrame(data = {'Cities': [x['Cities']]*len(price), 'Day_pre': [date_pre]*len(price), 'Day_after': [date_after]*len(price), 'location': location, 'price': price, 'rating': rating, 'info': _info, 'features': _features})

        df_here.to_csv(f"Data/airbnb_{x['Cities']}_{date_pre}_{date_after}.csv", index=True)

        driver.close()

        print(f"Done in {time.time() - time_start} seconds")

        return df_here 

    except Exception as e:
        print(e)
        print("error here")
        
        driver.close()
        print(f"Error in {x['Cities']} on {x['Day_pre'].date()} to {x['Day_after'].date()}")



In [5]:
# BARCELONA, KRAKOW, ROME, ATHEN, BOLOGNA, FIRENZE
# ROM FIRENSE PARIS ATHEN
# 0      0     1     0  

CITY_INDEXES = list(df_dates.index) # all cities
CITY_INDEXES = [16, 23, 24, 22, 17, 18, 25] # cities in last ran scraper
for i_d in CITY_INDEXES:
    i = 0
    while True and i < 5:
        i += 1
        try:
            df = get_row_prices(df_dates.iloc[i_d,:],  None) 
            
            if len(df['price']) > 50:  
                break
        except:
            print("error")
            time.sleep(1)
            
    time.sleep(60)

Getting prices for Barcalona on 2024-05-24 to 2024-05-25
https://www.airbnb.dk/s/Barcalona/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-09-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&date_picker_type=calendar&checkin=2024-05-24&checkout=2024-05-25&source=structured_search_input_header&search_type=filter_change
Found 126 stays in the range 0 - 435
Found 223 stays in the range 435 - 729
Found 168 stays in the range 729 - 1041
Found 220 stays in the range 1041 - 1423
Found 213 stays in the range 1423 - 1976
Found 175 stays in the range 1976 - 2486
Found 225 stays in the range 2486 - 4510
Found 12 stays in the range 4510 - 4822
skipped, not enough stays
total number of stays found 1350
0 done in thread
0 done in thread
0 done in thread
150 done in thread
150 done in thread
150 done in thread
300 done in thread
300 done in thread
300 done in thread
Done in 6702.7180235385895 sec